### Vehicle Detection

**Udacity SDCND Project 5**

In [21]:
# Import Needed Libraries
import os
import cv2
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# For HOG
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

from sklearn.model_selection import train_test_split

**1.0 HOG Feature Extraction**

Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier

In [15]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block,
                        vis=False, feature_vec=True):
    """
    Returns HOG features and optionally, visualization, 
    from an image using the sklearn library
    """
    
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True,
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True,
                       visualise=vis, feature_vector=feature_vec)
        return features

def extract_features(imgs, cspace='RGB', orient=9,
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    """
    Extract HOG features from a list of images, 
    with options of using different image colour spaces
    """
    
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel],
                                    orient, pix_per_cell, cell_per_block,
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient,
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Append the new feature vector to the features list
        features.append(hog_features)
    # Return list of feature vectors
    return features

In [16]:
# %% load data
# Vehicles
vehicles_path = 'vehicles/vehicles'
non_vehicles_path = 'non-vehicles/non-vehicles'


vehicles = [os.path.join(dirpath, f)
    for dirpath, dirnames, files in os.walk(vehicles_path)
    for f in files if f.endswith('.png')]

non_vehicles = [os.path.join(dirpath, f)
    for dirpath, dirnames, files in os.walk(non_vehicles_path)
    for f in files if f.endswith('.png')]

In [17]:
print vehicles[0]
print non_vehicles[0]

vehicles/vehicles\GTI_Far\image0000.png
non-vehicles/non-vehicles\Extras\extra1.png


In [35]:
colorspace = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"

t=time.time()

vehicle_features = extract_features(vehicles, cspace=colorspace, orient=orient,
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                        hog_channel=hog_channel)
non_vehicle_features = extract_features(non_vehicles, cspace=colorspace, orient=orient,
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                        hog_channel=hog_channel)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')

# Create an array stack of feature vectors
X = np.vstack((vehicle_features, non_vehicle_features)).astype(np.float64)

# Normalize features: by removing the mean and scaling to unit variance
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(vehicle_features)), np.zeros(len(non_vehicle_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

# Use a linear SVC
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# # Check the prediction time for a single sample
# t=time.time()
# n_predict = 10
# print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
# print('For these',n_predict, 'labels: ', y_test[0:n_predict])
# t2 = time.time()
# print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

D:\SciSoft\Anaconda2\lib\site-packages\skimage\feature\_hog.py:88: RuntimeWarning: invalid value encountered in sqrt
  image = np.sqrt(image)
D:\SciSoft\Anaconda2\lib\site-packages\skimage\feature\_hog.py:140: RuntimeWarning: invalid value encountered in remainder
  orientations, orientation_histogram)
D:\SciSoft\Anaconda2\lib\site-packages\skimage\feature\_hog.py:187: RuntimeWarning: invalid value encountered in double_scalars
  normalised_blocks[y, x, :] = block / np.sqrt(block.sum() ** 2 + eps)


(101.06, 'Seconds to extract HOG features...')


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

**1.1 Append Features**

[Optional] Apply a color transform and append binned color features, as well as histograms of color, to HOG feature vector. 

*Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.*

**2.0 Sliding Window Search**

Implement a sliding-window technique and use trained classifier to search for vehicles in images.

**3.0 Test Video test**

Run your pipeline on a video stream (starting with the test_video.mp4 and then full project_video.mp4) and 

Create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.

**4.0 Bounding Box Estimation**

Estimate a bounding box for vehicles detected.